In [35]:
from os import path
import importlib

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
torch.manual_seed(1)

import methodes
from methodes import Tokenizer
importlib.reload(methodes)

<module 'methodes' from 'c:\\Users\\stefa\\Documents\\workspaces\\szx81\\src\\methodes.py'>

In [36]:
from test_data import set_test_data, VALUE
from methodes import Tokenizer

set_test_data(
    data_count=10000, 
    moving_av=True)

tokenizer = Tokenizer(VALUE)
tokenizer.set_quantization_limits() 
shift = 0
window = 120 
time_temp_value = []
while shift + window < len(VALUE):
    time_temp_value.extend(tokenizer.get_sentence(VALUE[shift: shift + window]))
    shift += window

Test data size (flats are duducted) is 5754
Test data start time is 2023:01:03 21:34
Test data end time is   2023:01:09 21:39
Subtracting moving avarage: True


TypeError: object of type 'NoneType' has no len()

In [ ]:
from core import DATA_STORE

len(time_temp_value)
whole_story = Tokenizer.get_sentence_str(time_temp_value)
with open(
    path.join(DATA_STORE, f'whole_story_{Tokenizer.window}_{Tokenizer.number_pieces}.txt'), 'w') as f:
    for word in whole_story:
        f.write(f'{str(word)}\n')

In [2]:
words = Tokenizer.get_words_from_file()
print('len(words): ', len(words))
print('words[:3]:\n', words[:3])

len(words):  634
words[:3]:
 [('6516', 1736), ('6506', 1617), ('6606', 1412)]


Większość słów ma niewiele powtórzeń: te słowa zastąpię jednym: '0000'.

In [3]:
vocabulary = [_[0] for _ in words if _[1] > words[0][1] * 0.05]

print('len(vocabulary): ', len(vocabulary))
print('vocabulary[:3]:\n', vocabulary[:3])

len(vocabulary):  151
vocabulary[:3]:
 ['6516', '6506', '6606']


Wszystkie dane historyczne przepisuję na słowa mam teraz ciąg zdarzeń który podzielę na części do treningu i do testów.

In [4]:
org_story = Tokenizer.get_story_from_file()
story = [_ if _ in vocabulary else Tokenizer.none_word for _ in org_story]

print('len(story): ', len(story))
print('none-word count: ', story.count(Tokenizer.none_word))
print('story[:3]:\n', story[:3])

len(story):  62095
none-word count:  6703
story[:3]:
 ['4517', '7417', '6617']


In [ ]:
CONTEXT_SIZE = 5
EMBEDDING_DIM = 7

data_len = len(story) 
training_data = story[: (data_len // 3) * 2]
test_sentence = training_data
# Each tuple is ([ word_i-CONTEXT_SIZE, ..., word_i-1 ], target word)
ngrams = [
    (
        [test_sentence[i - j - 1] for j in range(CONTEXT_SIZE)],
        test_sentence[i]
    )
    for i in range(CONTEXT_SIZE, len(test_sentence))
]
# Print the first 3, just so you can see what they look like.
print('\nthe first 3 tuples of the training target:\n', ngrams[:3])

vocab = set(test_sentence)
word_to_ix = {word: i for i, word in enumerate(vocab)}

# https://pytorch.org/tutorials/beginner/nlp/word_embeddings_tutorial.html
# https://web.stanford.edu/~jurafsky/slp3/3.pdf
class NGramLanguageModeler(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs


losses = []
loss_function = nn.NLLLoss()
model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)

optimizer = optim.SGD(model.parameters(), lr=0.001)

for epoch in range(50):
    total_loss = 0
    for context, target in ngrams:

        # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
        # into integer indices and wrap them in tensors)
        context_idxs = torch.tensor([word_to_ix[w] for w in context], dtype=torch.long)

        # Step 2. Recall that torch *accumulates* gradients. Before passing in a
        # new instance, you need to zero out the gradients from the old
        # instance
        model.zero_grad()

        # Step 3. Run the forward pass, getting log probabilities over next
        # words
        log_probs = model(context_idxs)

        # Step 4. Compute your loss function. (Again, Torch wants the target
        # word wrapped in a tensor)
        loss = loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))

        # Step 5. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()

        # Get the Python number from a 1-element Tensor by calling tensor.item()
        total_loss += loss.item()
    print(f'epoch: {epoch}, total loss: {total_loss:.2e}')
    losses.append(total_loss)

print('\nloss decreased every iteration:\n', losses)  # The loss decreased every iteration over the training data!
# To get the embedding of a particular word, e.g. "beauty"


the first 3 tuples of the training target:
 [(['6617', '5617', '6617', '7417', '4517'], '5617'), (['5617', '6617', '5617', '6617', '7417'], '6617'), (['6617', '5617', '6617', '5617', '6617'], '6617')]
epoch: 0, total loss: 1.82e+05
epoch: 1, total loss: 1.61e+05
epoch: 2, total loss: 1.46e+05
epoch: 3, total loss: 1.37e+05
epoch: 4, total loss: 1.31e+05
epoch: 5, total loss: 1.27e+05
epoch: 6, total loss: 1.25e+05
epoch: 7, total loss: 1.23e+05
epoch: 8, total loss: 1.21e+05
epoch: 9, total loss: 1.19e+05
epoch: 10, total loss: 1.18e+05
epoch: 11, total loss: 1.17e+05
epoch: 12, total loss: 1.16e+05
epoch: 13, total loss: 1.15e+05
epoch: 14, total loss: 1.15e+05
epoch: 15, total loss: 1.14e+05
epoch: 16, total loss: 1.13e+05
epoch: 17, total loss: 1.13e+05
epoch: 18, total loss: 1.12e+05
epoch: 19, total loss: 1.12e+05
epoch: 20, total loss: 1.11e+05
epoch: 21, total loss: 1.11e+05
epoch: 22, total loss: 1.10e+05
epoch: 23, total loss: 1.10e+05
epoch: 24, total loss: 1.10e+05
epoch: 2

In [9]:
print('\nembedding the word "6617":\n', model.embeddings.weight[word_to_ix["6617"]])


embedding the word "6617":
 tensor([ 0.5322, -1.9512,  0.4350, -1.6504, -1.3422, -1.5822,  0.0048],
       grad_fn=<SelectBackward0>)


In [11]:
print(dir(model.embeddings))

['T_destination', '__annotations__', '__call__', '__class__', '__constants__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_apply', '_backward_hooks', '_backward_pre_hooks', '_buffers', '_call_impl', '_compiled_call_impl', '_fill_padding_idx_with_zero', '_forward_hooks', '_forward_hooks_always_called', '_forward_hooks_with_kwargs', '_forward_pre_hooks', '_forward_pre_hooks_with_kwargs', '_get_backward_hooks', '_get_backward_pre_hooks', '_get_name', '_is_full_backward_hook', '_load_from_state_dict', '_load_state_dict_post_hooks', '_load_state_dict_pre_hooks', '_maybe_warn_non_full_backward_hook', '_modules', '_named_members', '_non_persistent_buffers_set', '

In [ ]:
def make_context_vector(context, word_to_ix):
    idxs = [word_to_ix[w] for w in context]
    return torch.tensor(idxs, dtype=torch.long)

word_to_ix['6412']
# model()

40

In [25]:
test_data = story[data_len // 3:]
print(test_data[:11])

['6501', '6502', '6402', '6501', '0000', '6412', '6512', '5413', '7513', '6512', '0000']
